In [14]:
import warnings
import matplotlib.animation as animation 
from IPython import display 
import numpy as np
import h5py


In [15]:
#from phi.flow import *  # minimal dependencies
# from phi.torch.flow import *
# from phi.tf.flow import *

from phi.jax.flow import *

In [16]:
Res       = 200   # Resolution
Walls = Obstacle(union(
    [Box(x=(20, 60), y=(50, 51)), #bottom wall
    Box(x=(80, 100), y=(70, 71)), #top bar
    Box(x=(20, 21), y=(50, 65)),
    Box(x=(45, 46), y=(50, 65)),
    Box(x=(30, 31), y=(80, 100)),
    ]
))

@jit_compile  # Only for PyTorch, TensorFlow and Jax
def step(v, s, p, dt=1.):
    s = advect.mac_cormack(s, v, dt) + inflow
    buoyancy = resample(s * (0, 0.1), to=v)
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = fluid.make_incompressible(v, Walls, Solve('auto', 1e-4, 0, x0=p))
    return v, s, p


In [18]:
hf = h5py.File("Smoke-Flow-Data.h5", "w")


for source in np.linspace(10,90,9):

    print('source at ', source)
    
    
    velocity = StaggeredGrid(0, x=Res, y=Res, bounds=Box(x=100, y=100))  # or CenteredGrid(...)
    smoke    = CenteredGrid(0, ZERO_GRADIENT, x=2*Res, y=2*Res, bounds=Box(x=100, y=100))
    inflow   = 0.05 * resample(Sphere(x=source, y=10.0, radius=5), to=smoke, soft=True)
    pressure = None
    
    
    trajectory = [smoke]

    for _ in range(300):
        velocity, smoke, pressure = step(velocity, smoke, pressure)
        trajectory.append(smoke)
        #     plot(vis.overlay(smoke,Walls.geometry), alpha=1, show_color_bar=False, animate='time')
    trajectory = field.stack(trajectory, batch('time'))
    g = hf.create_group(name = 'x-{}'.format(source))
    g.create_dataset('velocity', data=trajectory.uniform_values().numpy('time,x,y,vector'))
    g.attrs['SourcePosition'] = source

hf.close()

source at  10.0
source at  20.0
source at  30.0
source at  40.0
source at  50.0
source at  60.0
source at  70.0
source at  80.0
source at  90.0


In [6]:
plot(vis.overlay(trj[1],Walls.geometry), alpha=1, show_color_bar=False, animate='time')

NameError: name 'trj' is not defined